In [1]:
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


from bs4 import BeautifulSoup


from selenium.webdriver import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")

In [3]:
# Initialize the browser
service = Service(ChromeDriverManager().install())
browser = Browser('chrome', service=service, options=chrome_options)

In [4]:
# Visit LinkedIn
url = "https://www.linkedin.com/feed/"
browser.visit(url)

# Wait for page to load
time.sleep(3)

In [5]:
# Get page title
page_title = browser.title
print(f"Page Title: {page_title}")


Page Title: LinkedIn Login, Sign in | LinkedIn


In [6]:


# If the login page appears, enter credentials
if "LinkedIn Login" in page_title or "Sign in | LinkedIn" in page_title:
    print("Login page detected. Entering credentials...")

    # Enter username
    browser.fill("session_key", "zacharysteele8@gmail.com")  # Replace with your email

    # Enter password
    browser.fill("session_password", "!Dragonblood90")  # Replace with your password

    # Find and click the Sign In button using its class
    sign_in_button = browser.find_by_css("button.btn__primary--large").first
    sign_in_button.click()

    # Wait for login
    time.sleep(5)

    # Check if login was successful
    print(f"New Page Title: {browser.title}")

else:
    print("Already logged in!")


# Wait to ensure the search results page loads
time.sleep(3)


Login page detected. Entering credentials...
New Page Title: Feed | LinkedIn


In [7]:
# Locate the search input field and enter a search term
search_input = browser.find_by_css("input.search-global-typeahead__input").first
search_input.fill("Data Engineer")

# Press Enter to submit the search
search_input.type("\n")

time.sleep(3)

In [8]:



jobs_button = browser.find_by_text("Jobs").first

# Click the "Jobs" button
jobs_button.click()

In [9]:


from selenium.webdriver.common.keys import Keys
import time

# Locate the job title input field
title_input = browser.find_by_xpath("//*[starts-with(@id, 'jobs-search-box-keyword-id-ember')]").first

# Locate the location input field
location_input = browser.find_by_xpath("//*[starts-with(@id, 'jobs-search-box-location-id-ember')]").first

# Ensure both fields are found before proceeding
if title_input and location_input:
    # Fill in "Data Engineer" in the title field
    title_input.fill("Data Engineer")
    print("Job title set to Data Engineer.")

    # Fill in "Tokyo" in the location field
    location_input.fill("Tokyo")
    print("Location set to Tokyo.")

    # Wait a moment for UI updates
    time.sleep(1)

    # Press Enter on one of the fields (location or title) to trigger search
    location_input._element.send_keys(Keys.ENTER)
    print("Search triggered.")

else:
    print("One or both input fields not found.")
    
# Wait for results to update
time.sleep(3)



Job title set to Data Engineer.
Location set to Tokyo.
Search triggered.


In [16]:
from bs4 import BeautifulSoup
import time

all_jobs = [] 

# Function to scroll inside job container
# def scroll_jobs():
#     scrollable_div = browser.find_by_css("#main > div > div.scaffold-layout__list-detail-inner.scaffold-layout__list-detail-inner--grow > div.scaffold-layout__list > div").first
#     last_height = browser.evaluate_script("arguments[0].scrollHeight", scrollable_div._element)

#     while True:
#         browser.execute_script("arguments[0].scrollTop += 500;", scrollable_div._element)  # Scroll down
#         time.sleep(2)  # Wait for jobs to load
#         new_height = browser.evaluate_script("arguments[0].scrollHeight", scrollable_div._element)
#         if new_height == last_height:  # Stop when no new jobs load
#             break
#         last_height = new_height
#     print("Scrolling complete.")

def scroll_jobs():
    scrollable_div = browser.find_by_css("#main > div > div.scaffold-layout__list-detail-inner.scaffold-layout__list-detail-inner--grow > div.scaffold-layout__list > div").first
    
    # Scroll directly to the bottom
    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div._element)
    
    # Wait for jobs to fully load
    time.sleep(2)  

    print("Scrolling complete.")

# Wait for pagination buttons to load
time.sleep(2)
page_html = browser.html
soup = BeautifulSoup(page_html, "html.parser")

# Find all pagination buttons dynamically
pagination_buttons = browser.find_by_css("button[aria-label^='Page ']")
total_pages = len(pagination_buttons)  # Count total pages

print(f"Total pages found: {total_pages}")

# Extract the available page numbers in order
page_numbers = []
for button in pagination_buttons:
    try:
        page_number = int(button.text.strip()) 
        page_numbers.append(page_number)
    except ValueError:
        continue  

page_numbers = sorted(set(page_numbers))  

print(f"Pages to scrape: {page_numbers}")

# Iterate through all valid pages
for page_num in page_numbers:
    print(f"Scraping page {page_num}")

    # Scroll to load all jobs on the current page
    scroll_jobs()

    # Get the current page's HTML
    page_html = browser.html
    soup = BeautifulSoup(page_html, "html.parser")

    # Find the updated job list container
    job_list_ul = soup.select_one("#main > div > div.scaffold-layout__list-detail-inner.scaffold-layout__list-detail-inner--grow > div.scaffold-layout__list > div > ul")

    # Extract job titles & company names
    if job_list_ul:
        job_listings = job_list_ul.find_all("li", {"data-occludable-job-id": True})

        for job in job_listings:
            try:
                # Extract job title
                job_title_element = job.find("a", class_="job-card-container__link")
                job_title = job_title_element.get_text(strip=True) if job_title_element else "No Title Found"

                # Extract company name properly
                company_element = job.find("div", class_="artdeco-entity-lockup__subtitle")

                if company_element:
                    company_span = company_element.find("span")  # Get first span inside the div
                    if company_span:
                        company_name = company_span.get_text(strip=True)
                    else:
                        company_name = company_element.get_text(strip=True)  # Fallback if span is missing
                else:
                    company_name = "No Company Found"

                # Store job data
                if job_title != "No Title Found":
                    all_jobs.append((job_title, company_name))

            except Exception as e:
                print(f"Error extracting job details: {e}")

    print(f"Total jobs found on page {page_num}: {len(all_jobs)}")

    # If there's another page, click the corresponding button
    if page_num < max(page_numbers):
        try:
            next_page_button = browser.find_by_css(f"button[aria-label='Page {page_num + 1}']").first
            if next_page_button:
                print(f"Clicking Page {page_num + 1} button...")
                next_page_button.click()
                time.sleep(3)  # Wait for new page to load
            else:
                print(f"Page {page_num + 1} button not found. Stopping.")
                break
        except Exception as e:
            print(f"Error clicking Page {page_num + 1}: {e}")
            break

print("Scraping completed!")

# Print all collected jobs
for job_title, company in all_jobs:
    print(f"Job Title: {job_title} | Company: {company}")

print(f"Total jobs collected: {len(all_jobs)}")



Total pages found: 6
Pages to scrape: [1, 2, 3, 4, 5, 6]
Scraping page 1
Scrolling complete.
Total jobs found on page 1: 11
Clicking Page 2 button...
Scraping page 2
Scrolling complete.
Total jobs found on page 2: 23
Clicking Page 3 button...
Scraping page 3
Scrolling complete.
Total jobs found on page 3: 35
Clicking Page 4 button...
Scraping page 4
Scrolling complete.
Total jobs found on page 4: 47
Clicking Page 5 button...
Scraping page 5
Scrolling complete.
Total jobs found on page 5: 59
Clicking Page 6 button...
Scraping page 6
Scrolling complete.
Total jobs found on page 6: 68
Scraping completed!
Job Title: Data EngineerData Engineer with verification | Company: TEKsystems
Job Title: Data EngineerData Engineer | Company: Aboitiz Data Innovation (ADI)
Job Title: Data Engineer, Japan Operations - Emerging MarketsData Engineer, Japan Operations - Emerging Markets with verification | Company: Amazon
Job Title: Senior Data EngineerSenior Data Engineer with verification | Company: Quant

In [15]:
all_jobs

[('Data EngineerData Engineer with verification', 'TEKsystems'),
 ('Data EngineerData Engineer', 'Aboitiz Data Innovation (ADI)'),
 ('Data Engineer, Japan Operations - Emerging MarketsData Engineer, Japan Operations - Emerging Markets with verification',
  'Amazon'),
 ('Senior Data EngineerSenior Data Engineer with verification', 'Quantexa'),
 ('Data Engineer - Search Department (SED)Data Engineer - Search Department (SED) with verification',
  'Rakuten'),
 ('Data Engineer, Credit ModelingData Engineer, Credit Modeling with verification',
  'PayPay Corporation'),
 ('Data Engineer / Data Analyst - Ad & Media Solutions Section, Data Solutions Department, Marketing Solutions Business (C&M)Data Engineer / Data Analyst - Ad & Media Solutions Section, Data Solutions Department, Marketing Solutions Business (C&M) with verification',
  'Rakuten'),
 ('Senior Data EngineerSenior Data Engineer with verification', 'TEKsystems'),
 ('Data Engineer - EC Logistics Development Department (ECLD)Data Eng

In [12]:
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys

all_jobs = []  # Store job listings from all pages

# Function to scroll inside job container
def scroll_jobs():
    scrollable_div = browser.find_by_css("#main > div > div.scaffold-layout__list-detail-inner.scaffold-layout__list-detail-inner--grow > div.scaffold-layout__list > div").first
    last_height = browser.evaluate_script("arguments[0].scrollHeight", scrollable_div._element)

    while True:
        browser.execute_script("arguments[0].scrollTop += 500;", scrollable_div._element)  # Scroll down
        time.sleep(2)  # Wait for jobs to load
        new_height = browser.evaluate_script("arguments[0].scrollHeight", scrollable_div._element)
        if new_height == last_height:  # Stop when no new jobs load
            break
        last_height = new_height
    print("Scrolling complete.")

# Wait for pagination buttons to load
time.sleep(2)
page_html = browser.html
soup = BeautifulSoup(page_html, "html.parser")

# Find all pagination buttons dynamically
pagination_buttons = browser.find_by_css("button[aria-label^='Page ']")
total_pages = len(pagination_buttons)  # Count total pages

print(f"Total pages found: {total_pages}")

# Extract the available page numbers in order
page_numbers = []
for button in pagination_buttons:
    try:
        page_number = int(button.text.strip())  # Extract page number
        page_numbers.append(page_number)
    except ValueError:
        continue  # Skip non-numeric buttons

page_numbers = sorted(set(page_numbers))  # Ensure unique & sorted

print(f"Pages to scrape: {page_numbers}")

# Iterate through all valid pages
for page_num in page_numbers:
    print(f"Scraping page {page_num}")

    # Scroll to load all jobs on the current page
    scroll_jobs()

    # Get the current page's HTML
    page_html = browser.html
    soup = BeautifulSoup(page_html, "html.parser")

    # Find the updated job list container
    job_list_ul = soup.select_one("#main > div > div.scaffold-layout__list-detail-inner.scaffold-layout__list-detail-inner--grow > div.scaffold-layout__list > div > ul")

    # Extract job titles from the list
    if job_list_ul:
        job_titles = [
            tag.find("strong").get_text(strip=True)
            for tag in job_list_ul.find_all("a", class_="job-card-container__link")
            if tag.find("strong")
        ]
        for job in job_titles:
            all_jobs.append((job, "Unknown Company"))  # Company name logic can be added if available

    print(f"Total jobs found on page {page_num}: {len(job_titles)}")

    # If there's another page, click the corresponding button
    if page_num < max(page_numbers):
        try:
            next_page_button = browser.find_by_css(f"button[aria-label='Page {page_num + 1}']").first
            if next_page_button:
                print(f"Clicking Page {page_num + 1} button...")
                next_page_button.click()
                time.sleep(3)  # Wait for new page to load
            else:
                print(f"Page {page_num + 1} button not found. Stopping.")
                break
        except Exception as e:
            print(f"Error clicking Page {page_num + 1}: {e}")
            break

print("Scraping completed!")

# Print all collected jobs
for job_title, company in all_jobs:
    print(f"Job Title: {job_title} | Company: {company}")

print(f"Total jobs collected: {len(all_jobs)}")

# Save results to CSV
# df = pd.DataFrame(all_jobs, columns=["Job Title", "Company"])
# df.to_csv("linkedin_jobs.csv", index=False)
# print("Saved to linkedin_jobs.csv")


Total pages found: 6
Pages to scrape: [1, 2, 3, 4, 5, 6]
Scraping page 1
Scrolling complete.
Total jobs found on page 1: 25
Clicking Page 2 button...
Scraping page 2
Scrolling complete.
Total jobs found on page 2: 25
Clicking Page 3 button...
Scraping page 3
Scrolling complete.
Total jobs found on page 3: 25
Clicking Page 4 button...
Scraping page 4
Scrolling complete.
Total jobs found on page 4: 25
Clicking Page 5 button...
Scraping page 5
Scrolling complete.
Total jobs found on page 5: 25
Clicking Page 6 button...
Scraping page 6
Scrolling complete.
Total jobs found on page 6: 9
Scraping completed!
Job Title: Data Engineer | Company: Unknown Company
Job Title: Data Engineer | Company: Unknown Company
Job Title: Data Engineer, Japan Operations - Emerging Markets | Company: Unknown Company
Job Title: Senior Data Engineer | Company: Unknown Company
Job Title: Data Engineer - Search Department (SED) | Company: Unknown Company
Job Title: Data Engineer, Credit Modeling | Company: Unknown C

In [10]:
from bs4 import BeautifulSoup
import time

# Locate the scrollable job listings container
scrollable_div = browser.find_by_css("#main > div > div.scaffold-layout__list-detail-inner.scaffold-layout__list-detail-inner--grow > div.scaffold-layout__list > div").first

# JavaScript to scroll inside the div
last_height = browser.evaluate_script("arguments[0].scrollHeight", scrollable_div._element)

while True:
    browser.execute_script("arguments[0].scrollTop += 500;", scrollable_div._element)  # Scroll down
    time.sleep(2)  # Wait for jobs to load
    new_height = browser.evaluate_script("arguments[0].scrollHeight", scrollable_div._element)

    if new_height == last_height:  # Stop when no new jobs load
        break
    
    last_height = new_height

print("Scrolling complete. Extracting job titles...")

# **Get updated HTML after scrolling**
page_html = browser.html
soup = BeautifulSoup(page_html, "html.parser")

# **Find the updated job list**
job_list_ul = soup.select_one("#main > div > div.scaffold-layout__list-detail-inner.scaffold-layout__list-detail-inner--grow > div.scaffold-layout__list > div > ul")

# **Extract job titles only from this <ul>**
if job_list_ul:
    job_titles = [
        tag.find("strong").get_text(strip=True)
        for tag in job_list_ul.find_all("a", class_="job-card-container__link")
        if tag.find("strong")
    ]

    # **Print job titles**
    for job in job_titles:
        print(job)
else:
    print("Job list container not found.")


Scrolling complete. Extracting job titles...
Data Engineer
Data Engineer
Data Engineer, Japan Operations - Emerging Markets
Senior Data Engineer
Data Engineer - Search Department (SED)
Data Engineer, Credit Modeling
Data Engineer / Data Analyst - Ad & Media Solutions Section, Data Solutions Department, Marketing Solutions Business (C&M)
Senior Data Engineer
Data Engineer - EC Logistics Development Department (ECLD)
Data Engineer
Data Engineer
Image Data Engineer
Data Engineer - Cashalo/Empower Asia Pacific (Remote)
Senior Data Engineer
Data Engineer FoundationX
Data Engineer - Business Data Solution Group, Business Support Development Department (BSD)
Lead, Data & ETL Engineer, Global DD&T(Data, Digital & Technology), Japan
Data Engineer (Consultant)
Clinical Data Engineer RWD
Azure Data Engineer - Life Insurance Company
Data Engineer / Support Engineer
Senior Data Engineer -AI & DATA Office (RMI AI & Data Div)
Data Security Specialist/Engineer
Data Engineer, Machine Learning Developme

In [11]:
stop

NameError: name 'stop' is not defined

In [ ]:

from bs4 import BeautifulSoup
import time
import pandas as pd

all_jobs = []  # Store job listings from all pages

# Wait for pagination buttons to load
time.sleep(3)
page_html = browser.html
soup = BeautifulSoup(page_html, "html.parser")

# Find all pagination buttons dynamically
pagination_buttons = browser.find_by_css("button[aria-label^='Page ']")
total_pages = len(pagination_buttons)  # Count total pages

print(f"Total pages found: {total_pages}")

# Extract the available page numbers in order
page_numbers = []
for button in pagination_buttons:
    try:
        page_number = int(button.text.strip())  # Extract page number
        page_numbers.append(page_number)
    except ValueError:
        continue  # Skip non-numeric buttons

page_numbers = sorted(set(page_numbers))  # Ensure unique & sorted

print(f"Pages to scrape: {page_numbers}")

# Iterate through all valid pages
for page_num in page_numbers:
    print(f"Scraping page {page_num}")

    # Wait for jobs to load
    time.sleep(3)

    # Get the current page's HTML
    page_html = browser.html
    soup = BeautifulSoup(page_html, "html.parser")

    # Find all job listings directly
    job_listings = soup.find_all("li", {"data-occludable-job-id": True})

    print(f"Total jobs found on page {page_num}: {len(job_listings)}")

    # Extract job titles and company names
    for job in job_listings:
        print(f'Job{job}')
        try:
            # Extract job title
            job_title_element = job.find("a", class_="job-card-container__link")
            job_title = job_title_element.get_text(strip=True) if job_title_element else "No Title Found"

            # Extract company name
            company_element = job.find("span", class_="artdeco-entity-lockup__subtitle")
            company = company_element.get_text(strip=True) if company_element else "No Company Found"

            # Store only jobs that have a title
            if job_title != "No Title Found":
                all_jobs.append((job_title, company))

        except Exception as e:
            print(f"Error extracting job: {e}")

    # If there's another page, click the corresponding button
    if page_num < max(page_numbers):
        try:
            next_page_button = browser.find_by_css(f"button[aria-label='Page {page_num + 1}']").first
            if next_page_button:
                print(f"Clicking Page {page_num + 1} button...")
                next_page_button.click()
                time.sleep(3)  # Wait for new page to load
            else:
                print(f"Page {page_num + 1} button not found. Stopping.")
                break
        except:
            print(f"Error clicking Page {page_num + 1}.")
            break

print("Scraping completed!")

# Print all collected jobs
for job_title, company in all_jobs:
    print(f"Job Title: {job_title} | Company: {company}")

print(f"Total jobs collected: {len(all_jobs)}")

# Save results to CSV
df = pd.DataFrame(all_jobs, columns=["Job Title", "Company"])
df.to_csv("linkedin_jobs.csv", index=False)
print("Saved to linkedin_jobs.csv")



In [ ]:
# Close the browser
browser.quit()